In [1]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

In [2]:
import os
os.environ['LANGCHAIN_PROJECT'] = 'Runnable'

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


## Langchain에서 데이터를 효과적으로 전달하는 방법
1. RunnablePassthrough
2. RunnableParallel
3. RunnableLambda


### 1. `RunnablePassthrough`: 데이터를 그대로 넘겨(통과시켜)주는 역할

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{word}를 영어로?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm
chain

PromptTemplate(input_variables=['word'], input_types={}, partial_variables={}, template='{word}를 영어로?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f1d56591060>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f1d56593160>, root_client=<openai.OpenAI object at 0x7f1d706fcf40>, root_async_client=<openai.AsyncOpenAI object at 0x7f1d565910c0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

chain 을 `invoke()` 하여 실행할 때는 입력 데이터의 타입은 ***딕셔너리***

In [5]:
# chain 을 실행합니다.
chain.invoke({"word": '사과'})

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4b1ee476-43f4-40a9-95fe-6546b6404171-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

But, 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능

In [6]:
# chain 을 실행합니다.
chain.invoke('사과')

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49efe1c5-f679-485a-9268-beaa525c5ab9-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [7]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough 는 runnable 객체이며, runnable 객체는 invoke() 메소드를 사용하여 별도 실행이 가능
# RunnablePassthrough()를 사용하여 체인 구성
runnable_chain = {"word": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경됨
runnable_chain.invoke('인공지능')

AIMessage(content='Artificial Intelligence (AI)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 17, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-917378fc-3f86-4bde-8f9f-30c180b4593e-0', usage_metadata={'input_tokens': 17, 'output_tokens': 6, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 2. `RunnableParallel`: 여러 작업을 동시에(병렬)로 처리하도록 도와주는 도구

In [8]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [9]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-017d859a-95e4-4361-8829-4af71ebaf45d-0', usage_metadata={'input_tokens': 19, 'output_tokens': 5, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 20, 'total_tokens': 42, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'g

### 3. `RunnableLambda`: 사용자 정의 함수 매핑하도록 도와주는 도구


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def concat_output(text):
    return text['capital'].content + ' ' + text['area'].content

In [12]:
final_chain = (combined_chain
            | {'info': RunnableLambda(concat_output)} 
            | PromptTemplate.from_template("{info}의 내용을 자연스럽게 다듬고, 이모지를 넣어줘.")
            | ChatOpenAI(model_name='gpt-4o-mini'))

final_chain.invoke("대한민국")

AIMessage(content='서울특별시는 대한민국의 수도로, 면적은 총 100,364㎢에 달합니다. 🇰🇷🌆', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 44, 'total_tokens': 73, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-25ad2b99-0d6d-4b9b-ac77-dc1bc2fb9222-0', usage_metadata={'input_tokens': 44, 'output_tokens': 29, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [14]:
from langchain_teddynote.messages import stream_response

answer = final_chain.stream('대한민국')
stream_response(answer)

안녕하세요! 서울입니다. 🇰🇷 대한민국의 면적은 총 100,363㎢입니다. 🌍✨

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Oct-14'

In [16]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 대한민국 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
# 출력
print(chain.invoke(3))

다음은 10월 14일에 생일인 대한민국의 유명인 3명입니다:

1. **이정재** - 1973년 10월 14일
2. **김범수** - 1979년 10월 14일
3. **이상민** - 1973년 10월 14일

이 외에도 다른 유명인들이 있을 수 있지만, 위의 세 명이 대표적인 인물입니다.


### [실습] Runnnable 세가지를 적절히 사용하여 챗봇 생성

- 위 코드를 활용하여 아래 내용에 맞게 작성하시요.
- RunnableParallel 사용
    - chain1 : `{food}` 재료로 만들 수 있는 메인 디시를 묻는 체인
    - chain2 : `{food}` 재료로 만들 수 있는 후식 디저트를 묻는 체인
- RunnableLambda 사용
    - 두 체인의 결과 값 이어 붙이기 -> `{info}`
    - `{info}`를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿


In [20]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 첫 번째 체인: 재료로 만들 수 있는 메인 디시를 묻는 체인
chain1 = (
    {'ingredient':RunnablePassthrough()}
    | PromptTemplate.from_template("{ingredient}로 만들 수 있는 요리는?")
    | ChatOpenAI()
)

# 두 번째 체인: 재료로 만들 수 있는 후식 디저트를 묻는 체인
chain2 = (
    {'ingredient':RunnablePassthrough()}
    | PromptTemplate.from_template("{ingredient}로 만들 수 있는 후식 또는 디저트는?")
    | ChatOpenAI()
)

# 두 체인을 병렬로 실행
combined_chain = RunnableParallel(dish=chain1, dessert=chain2)

# 두 체인의 결과를 이어 붙이는 함수
def concat_food_info(text):
    return text['dish'].content + ' '+ text['dessert'].content

# {info}를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿
prompt = PromptTemplate.from_template("{info}의 내용을 바탕으로 이모지를 사용하여 우리 음식점 메뉴의 추천 홍보문구를 작성해줘.")

# 최종 체인은 두 체인의 결과를 이어 붙이고 우리 가게 홍보 문구를 생성하는 작업
final_chain = (combined_chain
            |{'info':RunnableLambda(concat_food_info)}
            |prompt
            |ChatOpenAI(model_name='gpt-4o-mini')

)

# 사용자가 생성할 메뉴의 개수를 입력받아 처리
final_output = final_chain.invoke({'food':'토마토'})

# 결과 출력
print(final_output)


content='🍅✨ 신선한 토마토의 매력을 만끽하세요! ✨🍝 \n\n1. 🍝 **토마토 소스 파스타** - 달콤한 토마토 소스와 함께하는 파스타의 진수!\n2. 🥗 **그리스 샐러드** - 신선한 채소와 올리브로 건강함을 더한 한 접시!\n3. 🍲 **토마토 수프** - 따뜻하고 부드러운 토마토 수프 한 그릇으로 힐링하세요!\n4. 🍅🥗 **토마토 카프레제 샐러드** - 신선한 모짜렐라와 바질로 완성된 이탈리안 클래식!\n5. 🍝 **토마토 스파게티** - 풍부한 맛의 조화, 누구나 사랑하는 스파게티!\n6. 🥗 **토마토 샐러드** - 신선한 토마토의 상큼함을 느껴보세요!\n7. 🍞 **토마토 파니니** - 바삭한 빵 사이에 담긴 토마토의 풍미!\n8. 🍚 **토마토 리조또** - 크리미한 리조또에 토마토의 맛을 더해보세요!\n9. 🥪 **토마토 베이컨 샌드위치** - 고소한 베이컨과 신선한 토마토의 완벽한 조화!\n10. 🍲 **토마토 감자 수프** - 감자와 토마토의 부드러운 만남!\n\n🌟 그리고 디저트도 놓치지 마세요! 🌟\n\n1. 🍹 **토마토 스무디** - 상큼한 한 모금으로 기분 전환!\n2. 🍰 **토마토 케이크** - 색다른 디저트로 특별한 시간을 만들어보세요!\n3. 🍦 **토마토 아이스크림** - 신선한 토마토로 만든 이색 아이스크림!\n4. 🥣 **토마토 요거트 파르페** - 건강하고 맛있는 한 끼를 완성하세요!\n\n🍽️ 다양한 토마토 요리로 가득한 우리 음식점에서 특별한 맛의 여행을 떠나보세요! 🍅❤️' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 506, 'prompt_tokens': 256, 'total_tokens': 762, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {

In [ ]:
final_output